In [1]:
# import all function needed

import numpy as np
import re,copy,os,json
import subprocess 


# from calculator.gromacs.gromacs   import *
from calculator.gromacs.mdp_write import print_default_mdp
# from calculator.gromacs.pdb_gro   import *
# from calculator.gromacs.top_itp   import *
from calculator.gromacs.gromacs_workflow import *
# from calculator.gromacs.logger import *


# General md simulaiton test

This test case is a sample workflow of a normal md simualiton with gromacs. In the workflow, tasks of different types are involved(minimization, npt, nvt, anneal, normal_node), and they are generally used in simulaiton studies. By combining task steps of different task types, a md simulation protocol is realized and applied to the Remdesivir crystal structre.

There are five md steps in the protocol, and those steps are as bellow:

1.minimization
Energy minimization of the initial structure

2.nvt
NVT ensemble simulation

3.anneal
Anneal the structure from low temperature to target tempurature

4.npt
Relax structure

5.normal_mode\n
Caculate the frequency of structure

The perpose of the protocol is to calculate the frequency of structure at certain tempurature, and with the potential energy of strucure, a quasi-harmoic approximation free energy of remdesivir crystal structre at the target tempurature is obtained. 

If the protocl is built, which can be used to calculate free energy of other system conviently.

## mdp parameter 

The mdp parameter for task steps is a dict, the key and value in it corresponding to item and value in mdp file.
There are a list of default mdp parameters for supported task types, only key mdp items and values are needed，so mdp parameter dict can be short. 

If you want to know the mdp file contents for certain task types, you can used the function 
print_default_mdp from calculator.gromacs.top_itp, print mdp file content and check the parameters

In [2]:
print(print_default_mdp("nvt", {"nsteps": 5000}))

; RUN CONTROL PARAMETERS 
integrator               =  sd
tinit                    =  0
dt                       =  0.001
nsteps                   =  5000
comm-mode                =  Linear

; OUTPUT CONTROL OPTIONS
nstlog                   =  1000
nstenergy                =  1000
nstcalcenergy            =  1
nstxout                  =  0
nstxout-compressed       =  10000
compressed-x-precision   =  10000

; NEIGHBORSEARCHING PARAMETERS 
cutoff-scheme            =  Verlet
nstlist                  =  10
ns-type                  =  grid
pbc                      =  xyz
rlist                    =  1.2

; OPTIONS FOR ELE AND VDW
coulombtype              =  PME-Switch
rcoulomb-switch          =  1.18
rcoulomb                 =  1.2
vdwtype                  =  PME
vdw-modifier             =  Potential-Shift
rvdw                     =  1.2
DispCorr                 =  no
fourierspacing           =  0.1
pme-order                =  4
ewald-rtol               =  1e-05
ewald-geometry           =  3

In [3]:
# madp parameter for test 1

nvt_mdp = {"nsteps": 5000, 
           "nstxout-compressed": 100, 
           "compressed-x-precision":10000, }

anneal_mdp = {'compressed-x-precision': 10000,
              'nsteps': 5000,
              'nstxout-compressed': 100,
              'annealing-temp': '300 310',
              'annealing-time': '0 10',}

npt_mdp = {'nsteps': 5000, 
             'compressed-x-precision': 10000, 
             'nsteps': 5000, 
             'nstxout-compressed': 100}

## Forcefield and structure

The force field and structure are read as files, gro file and itp file are supported. The names of molecules in structures and force field files should be same, and the names of different molecules should be different.

In [4]:
# force field and structure for input 

# read the force field files
ff_list = [os.path.abspath("./files/1.general/LIG.itp")]

# read the structure, gro or g96 files supported
structure = os.path.abspath("./files/1.general/conf.gro")



## Input dict for workflow 

The workflow dict includes all parameters needed for gromacs md simulaiton. Those paramters have three parts, the task step name list, task step parameters and the start mode.

In [9]:

workflow_json = { 
    
    # Names of task steps, those task steps will be conducted one by one sequentially
    "task_step" :["1.minimization", "2.nvt", "3.anneal", "4.npt", "5.normal_mode"],
    
    #           parameters for task step.
    #           runtype : to use openmpi, ntmpi, ntomp or gpu to run the simulaiton tasks,
    #                     Supported keywords are "ntomp", "openmpi","ntmpi" ,"gpu", or mixture 
    #                     of them,  like "gpu_ntmpi"( to used gpu and ntmpi sametime ). But "openmpi"
    #                     and "ntmpi" can not be presented, cause they are not compatible
    #           core_num： total thread num, which will be omitted when runtype is "openmpi_ntomp" 
    #                      or "ntmpi_ntmpi"
    #           mpi_num： thread num of openmpi or ntmpi 
    #           omp_num： thread num of openmp
    #           gpu_id：  value of -gpu_id item in "gmx mdrun " , indexes of gpu used in calculaiton
    
    #           task_type : task type of task step, supported keywords are ["minimization", "npt", "nvt", 
    #                       "replica_exchange", "free_energy", "pull","anneal","normal_mode", "rerun","cmd"].
    #                        There are default mdp parameters for every task types, except for cmd, which means 
    #                       to exccute cmd or shell conmmand lines
    #           cmd       : string or a list of string, the cmd or shell conmmand line strings
    #           ff_list   : force filed file list 
    #           ff_type   : force filed type, default is gaff, "gaff" and "opls" are supported, "gaff" is default 
    #           mdp_para  : mdp paramneter dict
    #           structure : input struture, gro file or gro file list. If not provided, the struture result of 
    #                       last step will be used. If it is a list of structure, the simulaiton will be done for
    #                       every structure in the list one by one
    #           pos_restraint : position restriant, it is a dict, the key of dict is the force field molecule 
    #                           name, and value is a list, the elements of list is the parameter of restriant 
    #                           settings, refer to free energy test eamxple for detail using
    

    #           ref_structure : reference structure path
    
    #           index     : parameter to generate index, ndx file. It is a dict, the keys of dict is the group 
    #           name, and the value is also a dict, the key of dict can be the molecule name of mol in input structure, 
    #           or keywords "mol" ,"atom", and the value is the list of index of molecules, mol or atom, number and 
    #           string like "1-20" are supported
    
    #           precision : the verison of gromacs used in simulaiton, double or mixed precision
    
    #           result    : result process, which is a dict, the key is the result type, and the value 
    #                       result type is the parameter for result process. Keywords of "rmsd", "structure", "energy", "wham",
    #                       "bar" types can be used. If result not presented, the final output structure
    #                       is returned as result.
    
    #                       for structure, dump_time, index and group can be added, dump_time is a list or a float
    #                       for rmsd, index and group can be added
    #                       for energy, item can be add, string or int, or a list of string  or int, which decide which energy part read.
    #                       If no item presented, all item will be returned
    #                       for wham and bar, no parameter needed
    
    
    
                "1.minimization":{"runtype":"ntomp","core_num":1,"task_type":"minimization",
                                  "ff_list":ff_list,
                                  "mdp_para":minimization, "structure": structure},
    
                "2.nvt"     :{"runtype":"ntomp", "task_type":"nvt", "ff_list":ff_list,
                                   "mdp_para":nvt_mdp, "result":{"rmsd":{}, "energy":{}}},
                "3.anneal"  :{"runtype":"ntomp", "task_type":"anneal","ff_list":ff_list,
                                  "mdp_para":anneal_mdp,"result":{"rmsd":{}, "energy":{}} },
                "4.npt"     :{"runtype":"ntomp", "task_type":"npt", "ff_list":ff_list,
                                   "mdp_para":nvt_mdp, "result":{"rmsd":{}, "energy":{}}},
                "5.normal_mode"  :{"runtype":"ntomp","core_num":4, "task_type":"normal_mode", "ff_list":ff_list,
                                   "mdp_para":{"integrator": "nm"}},
    
    # start_mode:  start mode of workflow, "from_scratch" is to start the workflow brand new, and backup files before,
    #               and "continue_run" is to continue the task unfinished before
    "start_mode" : "from_scratch",
    
    
    # start step index of workflow
    "start_step" : 0,

    
                }

## Gromacs exe path

Dict of gmx_exe includes the paths of gromacs executing files and mpirun, and the path of libraries needed. About how to install gromacs of different verisons, you can refer to the script gmx_install.sh file, in which all verison of gromacs used in the workflow are installed. If gromacs installed as the script, then the gmx_exe can be as follow 

In [ ]:
# openmpi install path
openmpi_base_path = "xxx_openmpi"

# gromacs install path
gromacs_basepath = "xxx_gromacs"

gmx_exe = {
    
    "mpirun":       openmpi_base_path + "/bin/mpirun",
    "openmpi_lib" : openmpi_base_path + "/lib",
          
    "gmx"         :  gromacs_basepath + "/sp/bin/gmx" , 
    "gmx_mpi"     :  gromacs_basepath + "/sp_mpi/bin/gmx_mpi" , 
    "gmx_d"       :  gromacs_basepath + "/dp/bin/gmx_d" , 
    "gmx_mpi_d"   :  gromacs_basepath + "/dp_mpi/bin/gmx_mpi_d" , 
    "gmx_gpu"     :  gromacs_basepath + "/gpu/bin/gmx" , 
    "gmx_gpu_mpi" :  gromacs_basepath + "/gpu_mpi/bin/gmx_mpi" , 
    
    }


## Run workflow

As the running of the simulaiton, log information of workflow will be saved and printed.

In [3]:
# all files of task steps will be save in run_dir


run_dir = "./1.general/"
os.mkdir(run_dir)
os.chdir(run_dir)

task = Gromacs_workflow(workflow_json=workflow_json, gmx_exe=gmx_exe)
task.task_run() 

# multi-subtask and rerun test

In this test case, a task step with multi-subtasks and a rerun of a former task step are included.

In the workflow protocol, at the step "3.anneal", structures of time [0.1,0.2,0.3,0.4,0.5] in trajctory are dumped, and taken as the input structure of next step, which responding to the structures at temperature of [302,304,306,308,310] in the anneal simulaiton. The input structures of step "4.npt" is a list, at same time the "ref-t" parameter in mdp is also a list. Then there will be five subtasks in "4.npt". For the first task, the input structure is the first structure in structure list, and the "ref-t" is first temperature in "ref-t" parameter list; For the second task, the structure and "ref-t" parameter in the list will be used, and so on. All other paramter are the same.

When simulaiton tasks which only serveal mdp parameters are different, or the input structures are different, those parameters or structure can be setted as a list in workflow imput dict, and those simulaiton tasks will be the subtasks of the step.

As the subtasks are calculating one by one, so it may take a long time to finish all the tasks, if there are many subtasks. Rather then using a larger core number in gromacs runing, here is a trick to speed up the calculation. Stopping the task step after all the gromacs file for running created, then all the tasks can be running at same time if we submit those subtasks to other hpc system. After finishing running all the subtasks, set the "start_mode" to "continue_run" and "start_step" to the index of current task step, and start the workflow protocol. The simulaiton will go on as all the subtasks finish and continue to the following result process part.


In the task type of task type is "rerun", the energies of trajctory in the task step we are to do rerun on will be calculated. If force file and mdp paramter, the parameter of those will same with the taget task step. Index is also supported.

In [24]:

ff_list = [os.path.abspath("./files/1.general/LIG.itp")]
structure = os.path.abspath("./test/1.general/conf.gro")


nvt_mdp["ref-t"] = 302

# The mdp parameter can be a list. When this is the case, simulaiton of every parameter in the list will 
# be conducted one by one. If the length of input structure list and the length of mdp parameter list is smae,
# then the simulation of the structure and the mdp parameter which have the same index in the structure and mdp parameter 
# list will be conducted,so there will be same number of sub-simuliaton task as the length of mdp parameter list.
# If the length of structure list is different from the length of mdp parameter list, a error will be raised, and 
# the simulaiton exits.So all length mdp parameter lists and structure list should be same
npt_mdp["ref-t"] = [302,304,306,308,310]

workflow_json = { "task_step" :["1.minimization", "2.nvt","3.anneal", "4.npt","6.rerun"],
                "1.minimization":{"runtype":"ntomp", "task_type":"minimization","ff_list":ff_list,
                                  "mdp_para":minimization, "structure": structure},
                "2.nvt"     :{"runtype":"ntomp", "task_type":"nvt", "ff_list":ff_list,                              
                                   "mdp_para":nvt_mdp, "result":{"rmsd":{}, "energy":{}}},
                "3.anneal"  :{"runtype":"ntomp", "task_type":"anneal","ff_list":ff_list,
                              "structure": structure,
                                  "mdp_para":anneal_mdp,"result":{"rmsd":{}, "energy":{},
                                                                   "structure":{"dump_time":[0.1,0.2,0.3,0.4,0.5]}} },
                "4.npt"     :{"runtype":"ntomp", "task_type":"npt", "ff_list":ff_list,
                                   "mdp_para":npt_mdp, "result":{"rmsd":{}, "energy":{}}},
#                 "5.normal_mode"  :{"runtype":"ntomp", "task_type":"normal_mode", "ff_list":ff_list,
#                                    "mdp_para":{"integrator": "nm"}},
                "6.rerun"   :{"task_type":"rerun", "task_step":"4.npt"},
                 "start_step" : 0,
#                  "start_mode" :"continue_run"
                
                }


In [22]:
os.chdir("C:\\Users\\dianc\\Desktop\\jpy文档\\11.operator")

In [25]:
run_dir = "./2.subtask/"
# os.mkdir(run_dir)
os.chdir(run_dir)

task = Gromacs_workflow(workflow_json=workflow_json, gmx_exe=gmx_exe)
task.task_run() 

[2022-10-14 10:56:48][INFO] Task_step 1.minimization starts 
[2022-10-14 10:56:48][INFO] Task_step 1.minimization starts 
[2022-10-14 10:56:48][INFO] Task_step 1.minimization starts 
[2022-10-14 10:56:48][INFO] Task_step 1.minimization starts 
[2022-10-14 10:56:48][INFO] Task_step 1.minimization starts 
[2022-10-14 10:56:48][INFO] The task type is minimization, and mdp parameters is {'integrator': 'steep', 'emtol': 2.0, 'emstep': 0.01, 'nsteps': 80000, 'nstcomm': 2, 'nstxout': 0, 'nstxout-compressed': 1000, 'tcoupl': 'no', 'gen-vel': 'no', 'pcoupl': 'no', 'tau-p': 10.0} 
[2022-10-14 10:56:48][INFO] The task type is minimization, and mdp parameters is {'integrator': 'steep', 'emtol': 2.0, 'emstep': 0.01, 'nsteps': 80000, 'nstcomm': 2, 'nstxout': 0, 'nstxout-compressed': 1000, 'tcoupl': 'no', 'gen-vel': 'no', 'pcoupl': 'no', 'tau-p': 10.0} 
[2022-10-14 10:56:48][INFO] The task type is minimization, and mdp parameters is {'integrator': 'steep', 'emtol': 2.0, 'emstep': 0.01, 'nsteps': 8000

[2022-10-14 11:06:33][INFO] The task type is npt, and mdp parameters is {'integrator': 'sd', 'dt': 0.001, 'nsteps': 5000, 'nstxout': 0, 'nstxout-compressed': 100, 'ref-t': [302, 304, 306, 308, 310], 'tcoupl': 'Berendsen', 'tau-t': 1.0, 'gen-vel': 'yes', 'pcoupl': 'Berendsen', 'tau-p': 3.0, 'compressed-x-precision': 10000} 
[2022-10-14 11:06:34][INFO] The subtask md_0 of 4.npt starts 
[2022-10-14 11:06:34][INFO] The subtask md_0 of 4.npt starts 
[2022-10-14 11:06:34][INFO] The subtask md_0 of 4.npt starts 
[2022-10-14 11:06:34][INFO] The subtask md_0 of 4.npt starts 
[2022-10-14 11:06:34][INFO] The subtask md_0 of 4.npt starts 
[2022-10-14 11:10:52][INFO] The subtask md_0 of 4.npt finishes 
[2022-10-14 11:10:52][INFO] The subtask md_0 of 4.npt finishes 
[2022-10-14 11:10:52][INFO] The subtask md_0 of 4.npt finishes 
[2022-10-14 11:10:52][INFO] The subtask md_0 of 4.npt finishes 
[2022-10-14 11:10:52][INFO] The subtask md_0 of 4.npt finishes 
[2022-10-14 11:10:52][INFO] The subtask md_1 

[2022-10-14 11:29:27][INFO] The subtask md_3 of 6.rerun finishes 
[2022-10-14 11:29:27][INFO] The subtask md_3 of 6.rerun finishes 
[2022-10-14 11:29:27][INFO] The subtask md_3 of 6.rerun finishes 
[2022-10-14 11:29:27][INFO] The subtask md_3 of 6.rerun finishes 
[2022-10-14 11:29:27][INFO] The subtask md_3 of 6.rerun finishes 
[2022-10-14 11:29:27][INFO] The subtask md_4 of 6.rerun starts 
[2022-10-14 11:29:27][INFO] The subtask md_4 of 6.rerun starts 
[2022-10-14 11:29:27][INFO] The subtask md_4 of 6.rerun starts 
[2022-10-14 11:29:27][INFO] The subtask md_4 of 6.rerun starts 
[2022-10-14 11:29:27][INFO] The subtask md_4 of 6.rerun starts 
[2022-10-14 11:29:33][INFO] The subtask md_4 of 6.rerun finishes 
[2022-10-14 11:29:33][INFO] The subtask md_4 of 6.rerun finishes 
[2022-10-14 11:29:33][INFO] The subtask md_4 of 6.rerun finishes 
[2022-10-14 11:29:33][INFO] The subtask md_4 of 6.rerun finishes 
[2022-10-14 11:29:33][INFO] The subtask md_4 of 6.rerun finishes 
[2022-10-14 11:29:33

# free energy test

The test case is to calculate the solvation free energy of of CH4 in water. Parameter, structure and force field are from toturial by Justin A. Lemkul (http://www.mdtutorials.com/gmx/free_energy/index.html).

Free energy calculaiton is realized with the subtasks method

In [ ]:

ff_list = [os.path.abspath(i) for i in glob.glob("./test/3.free_energy/*itp")]
structure = os.path.abspath("./test/3.free_energy/conf.gro")


nvt_mdp["ref-t"] = 300
npt_mdp["ref-t"] = 300

free_energy_0 = copy.deepcopy(free_energy)

free_energy_0["init-lambda-state"] = [5,6,7]
free_energy_0["nsteps"] = 5000
free_energy_0["couple-moltype"] = "Met"

workflow_json = { "task_step" :["1.minimization", "2.nvt","3.npt", "4.free_energy"],
                "1.minimization":{"runtype":"ntomp", "task_type":"minimization","ff_list":ff_list,
                                  "mdp_para":minimization, "structure": structure},
                "2.nvt"     :{"runtype":"ntomp", "task_type":"nvt", "ff_list":ff_list,                              
                                   "mdp_para":nvt_mdp, },
                "3.npt"     :{"runtype":"ntomp", "task_type":"npt", "ff_list":ff_list,
                                   "mdp_para":npt_mdp, },
                "4.free_energy"  :{"runtype":"ntomp", "task_type":"free_energy", "ff_list":ff_list,
                                   "ff_type":"opls",
#                                    "structure": structure,
                                   "mdp_para":free_energy_0,
                                    "result": {"bar":{}}},
                 "start_step" : 0,
#                  "start_mode" :"from_scratch"
                
                }


In [3]:
# run_dir = "./3.free_energy/"
# os.mkdir(run_dir)
# os.chdir(run_dir)
# task = Gromacs_workflow(workflow_json=workflow_json, gmx_exe=gmx_exe)
# task.task_run() 

#  pull test

The test case is to calculate the change of free energy along the center of mass distance between of two CH4 molecues.

Calculaiton is realized uisng the subtasks method. Position restraint and index are applied in the case

In [4]:

ff_list = [os.path.abspath(i) for i in glob.glob("./test/4.pull/*itp")]
structure = os.path.abspath("./test/4.pull/conf.gro")


pull_0 = {}

pull_0["nsteps"] = 5000
pull_0["pull-group1-name"] = "Met1"
pull_0["pull-group2-name"] = "Met2"

pull_1 = {}
pull_1["nsteps"] = 5000
pull_1["pull-group1-name"] = "Met1"
pull_1["pull-group2-name"] = "Met2"
pull_1["pull-coord1-start"] = "yes"
pull_1["pull-coord1-dim"] = "Y Y Y"
pull_1["pull-coord1-rate"] = "0.0"

workflow_json = { "task_step" :["1.minimization", "2.nvt","3.npt", "4.pull", "5.pull"],
                "1.minimization":{"runtype":"ntomp", "task_type":"minimization","ff_list":ff_list,
                                  "mdp_para":minimization, "structure": structure},
                "2.nvt"     :{"runtype":"ntomp", "task_type":"nvt", "ff_list":ff_list,                              
                                   "mdp_para":nvt_mdp, },
                "3.npt"     :{"runtype":"ntomp", "task_type":"npt", "ff_list":ff_list,
                                   "mdp_para":npt_mdp, },
                "4.pull"  :{"runtype":"ntomp", "task_type":"pull", "ff_list":ff_list,
                                    "pos_restraint":{"Met1":[[1,1,1000,1000,1000]],"Met2":[[1,1,1000,1000,0]]},
                                   "ff_type":"opls", "index":{"Met1":{"mol":[1]},"Met2":{"mol":[2]}},
#                                    "structure": structure,
                            "result":{"structure":{"dump_time":[0, 0.1,0.2]}},
                                   "mdp_para":pull_0},
                "5.pull"  :{"runtype":"ntomp", "task_type":"pull", "ff_list":ff_list,
                                   "ff_type":"opls", "index":{"Met1":{"mol":[1]},"Met2":{"mol":[2]}},
                                   "mdp_para":pull_1,
                            "result":{"wham":{}}
                            },
                 
#                  "start_step" : 3,
#                  "start_mode" :"from_scratch"
                
                }


# dir_path = "C://Users//dianc//Desktop//jpy文档//11.operator/pull_test/"
gmx_exe  = { "gmx" :"gmx.exe", "gmx_d" :"gmx.exe" }

In [2]:
run_dir = "./4.pull/"
os.mkdir(run_dir)
os.chdir(run_dir)
task = Gromacs_workflow(workflow_json=workflow_json, gmx_exe=gmx_exe)
task.task_run() 